<a href="https://colab.research.google.com/github/alessandronascimento/BioMolComp/blob/main/P10/SiRNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNA de Interferência

Neste último Colab, vamos treinar um modelo para indicar se uma sequência de RNA é uma boa sequência para causar o **silenciamento** de um gene.

O RNA de interferência foi descoberto nos anos 1990 e rendeu um prêmio Nobel em 2006. De forma muito resumida, um fragmento de RNA cuja sequência seja complementar a uma parte de um mRNA pode se ligar ao mRNA. O resultado é o *silenciamento* do mRNA, isto é, ele deixa de ser traduzido em uma proteína. O fragmento de RNA que silencia o gene é chamado de *short interfering RNA* ou siRNA. Este mecanismo permite "*desligar*" a transcrição de um gene e é muito empregado para a pesquisa sobre a importância do gene em determinado contexto.

Uma vez que as sequências de RNA não são aleatórias, e que o RNA possui uma estrutura complexa, diferentes sequências de RNA apresentam eficiências diferentes cini siRNA e seria interessante ter uma ferramenta que selecionasse as melhores sequências para um siRNA.

# Deep Learning

Uma biblioteca de 2187 moléculas de siRNA foram testadas experimentalmente, cada uma possuindo 21 bases. Cada uma recebeu um *score* numérico (um valor entre 0 e 1) indicando o quão efetivo foi o silenciamento do gene alvo.

Vamos gerar um modelo para, a partir destes dados, estimar a qualidade da sequência como uma sequência para siRNA.

## Bibliotecas necessárias

In [ ]:
# Instalando e importando as bibliotecas necessárias

!pip3 install deepchem
!apt-get install subversion
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
import matplotlib.pyplot as plot

!nvidia-smi

## Baixando os dados e carregando no ambiente

In [ ]:
# Get the data and load into deepchem

!svn export https://github.com/deepchem/DeepLearningLifeSciences/trunk/Chapter06/train_siRNA
!svn export https://github.com/deepchem/DeepLearningLifeSciences/trunk/Chapter06/valid_siRNA

# Load the data.

train = dc.data.DiskDataset('train_siRNA')
valid = dc.data.DiskDataset('valid_siRNA')

## Inspecionando as características dos dados

In [ ]:
train

## Construção do Modelo

Aqui vamos usar mais uma vez uma rede convolutiva CNN. Novamente a rede será uma rede 1D, com duas camadas, seguida por uma rede densa totalmente conectada.

Uma diferença importante aqui é que nosso modelo é uma modelo de **regressão**. Isto é, estamos tentando prever a probabilidade (0-1) de que a sequência seja uma sequência capaz de silenciar o gene alvo, de acordo com o dado experimental.

Por isso, a função de perda agora é uma função de distância $L_2$, que minimiza a diferença média quadrada entre os valores preditos e os valores reais.

In [4]:
# Build the model.

features = tf.keras.Input(shape=(21, 4))
prev = features
for i in range(2):
    prev = layers.Conv1D(filters=10, kernel_size=10, activation=tf.nn.relu, padding='same')(prev)
    prev = layers.Dropout(rate=0.3)(prev)
output = layers.Dense(units=1, activation=tf.math.sigmoid)(layers.Flatten()(prev))
keras_model = tf.keras.Model(inputs=features, outputs=output)
model = dc.models.KerasModel(
    keras_model,
    loss=dc.models.losses.L2Loss(),
    batch_size=1000,
    model_dir='rnai')

## Métrica

Uma vez que nossa predição agora trata-se de uma regressão, podemos usar como métrica a correlação (de Pearson) entre os valores reais e os valores preditos. A correlação nos dará um número entre -1 (anti-correlação) e 1 (correlação perfeita), onde 0 indica a ausência de qualquer correlação.

In [ ]:
# Train the model, tracking its performance on the training and validation datasets.

metric = dc.metrics.Metric(dc.metrics.pearsonr, mode='regression')

xs=[]
ytrain=[]
yvalid=[]

for i in range(20):
    model.fit(train, nb_epoch=10)
    print("Iteration %d:" % (i+1))
    m = model.evaluate(train, [metric])
    auc = m['pearsonr']
    print("ROC AUC [train]: %7.3f" % auc)
    ytrain.append(auc)
    m = model.evaluate(valid, [metric])
    auc = m['pearsonr']
    print("ROC AUC [valid]: %7.3f" % auc)
    yvalid.append(auc)
    xs.append(i+1)
    print()

## Análise gráfica

Como no exemplo anterior, também vamos olhar graficamente para a evolução do treinamento do nosso modelo usando o pyplot.

In [ ]:
# Representação Gráfica da Evolução do Treinamento

#plot.style.use('seaborn-whitegrid')
plot.ylabel('Pearson r')
plot.xlabel('Iteration')
plot.plot(xs, ytrain, '-ok', label='train')
plot.plot(xs, yvalid, '-or', label='valid')
plot.legend(loc='lower right')